**Imports**

In [1]:
import subprocess
from IPython.display import Video

**Resolution Modification**

In [5]:
def change_resolution(input_video, output_video, width, height):
    ffmpeg_cmd = [
        'ffmpeg',
        '-i', input_video,
        '-vf', f'scale={width}:{height}',
        '-c:v', 'libx264',
        '-crf', '23',
        '-c:a', 'copy',
        output_video
    ]
    
    subprocess.run(ffmpeg_cmd)
    print(f"Video resized to {width}x{height}")

In [6]:
input_video = 'input.mov'
output_video = 'output.mov'
width = 320
height = 180
change_resolution(input_video, output_video, width, height)

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

Video resized to 320x180


[out#0/mov @ 0x135e36000] video:12683KiB audio:31863KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.880836%
frame=14315 fps=1522 q=-1.0 Lsize=   44939KiB time=00:09:56.37 bitrate= 617.3kbits/s speed=63.4x    
[libx264 @ 0x135e36ea0] frame I:149   Avg QP:18.59  size: 13318
[libx264 @ 0x135e36ea0] frame P:6219  Avg QP:23.08  size:  1336
[libx264 @ 0x135e36ea0] frame B:7947  Avg QP:28.00  size:   339
[libx264 @ 0x135e36ea0] consecutive B-frames: 17.1% 24.6%  5.7% 52.6%
[libx264 @ 0x135e36ea0] mb I  I16..4:  5.5% 51.3% 43.2%
[libx264 @ 0x135e36ea0] mb P  I16..4:  0.5%  2.1%  1.5%  P16..4: 26.4% 10.7%  8.5%  0.0%  0.0%    skip:50.3%
[libx264 @ 0x135e36ea0] mb B  I16..4:  0.1%  0.5%  0.5%  B16..8: 19.2%  3.6%  1.4%  direct: 1.3%  skip:73.4%  L0:34.6% L1:53.0% BI:12.4%
[libx264 @ 0x135e36ea0] 8x8 transform intra:50.8% inter:56.4%
[libx264 @ 0x135e36ea0] coded y,uvDC,uvAC intra: 70.6% 78.9% 58.6% inter: 11.8% 10.1% 3.0%
[libx264 @ 0x135e36ea0] i16 v,h,dc,p: 27% 29% 

In [4]:
print("\nConverted video:")
Video(output_video)


Converted video:


ValueError: To embed videos, you must pass embed=True (this may make your notebook files huge)
Consider passing Video(url='...')